<a href="https://colab.research.google.com/github/zb15/B2BNetworkWiki/blob/main/B2B_Network_Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions for Python package: B2B_Network_Wiki

Currently the package has 3 main functions:            

1.   choose_company(input_name, search_option='all')
2.   get_companies_network(QIDs, num_runs=(5,5,5,5))
3.   visualise_b2b_network(df)

The **choose_company()** allows user to search companies with various search options: exact match; starts with the input plus space; starts with the input plus comma plus space, or as default all options.
User can search and choose any number of companies, and will need to save the QIDs only to a list to use them in the next function.


The **get_companies_network(QIDs, num_runs=(5,5,5,5))** has 4 set of sub-functions (e.g., has_parent() and if it does then get_parent_info()) to query the different realations (parent, owned by, subsidiary and owner of) plus some cleaning functions. It returns the max 4 dataframes (p_df, ob_df, s_df, oo_df) (empty if it is requested to be 0).  

The **visualise_b2b_network()** function has 2 steps at the moment (that can easily be made into 1 function). The 1st step (with clean_and_join(p_df, ob_df, s_df, oo_df) function) is to clean the 4 datasets and join them together into a "final_df". The wikidata requires a lot of cleaning, handling duplicates etc. and comments are included within (the quite long) code. And the function that visualising the network from "final_df" dataframe (visualise_b2b_network()).









## importing libraries

In [ ]:
!pip install pyvis --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00


In [ ]:
!pip install pycountry --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install pycountry-convert --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 9.7 MB/s eta 0:00:00


In [ ]:
# Importing the modules
import requests
import random
import pandas as pd
import numpy as np

## FUNCTION 1. **choose_company(input_name, search_option='all')** with searchCompanyByName()

### function

In [ ]:
def searchCompanyByName(input_query, search_option):
    # Importing the modules
    import requests
    import random
    import pandas as pd
    import numpy as np
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    query = ''
    if search_option == 'all':
        # Constructing the SPARQL query
        query = f""" SELECT distinct ?item ?QID ?itemLabel ?itemDescription
        WHERE{{
          ?item rdfs:label ?itemLabel.
          FILTER(REGEX(?itemLabel, ?name )).
          VALUES ?name {{"^{input_query}$"@en "^{input_query} "@en "^{input_query}, "@en}} .
          FILTER(LANG(?itemLabel)="en").
          ?item wdt:P31 ?type .
          VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                                wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                                wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                                wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities

          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.
        #the next 3 lines will give the QID in the column QID
        #This query will bind the substring of ?uri starting from the 32nd character to a new variable ?code
        BIND (STR (?item) AS ?uri) .
        FILTER (STRSTARTS (?uri, "http://www.wikidata.org/entity/Q")) .
        BIND (SUBSTR (?uri, 32) AS ?QID) .

        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """
    elif search_option == 'exact':
        query = f""" SELECT distinct ?item ?QID ?itemLabel ?itemDescription
         WHERE{{
           ?item rdfs:label ?itemLabel.
           FILTER(REGEX(?itemLabel, ?name )).
           VALUES ?name {{"^{input_query}$"@en}} .
           FILTER(LANG(?itemLabel)="en").
           ?item wdt:P31 ?type .
           VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                                 wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                                 wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                                 wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities

           ?article schema:about ?item .
           ?article schema:inLanguage "en" .
           ?article schema:isPartOf <https://en.wikipedia.org/>.
         #the next 3 lines will give the QID in the column QID
         #This query will bind the substring of ?uri starting from the 32nd character to a new variable ?code
         BIND (STR (?item) AS ?uri) .
         FILTER (STRSTARTS (?uri, "http://www.wikidata.org/entity/Q")) .
         BIND (SUBSTR (?uri, 32) AS ?QID) .

         SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
         }}
         """
    elif search_option == 'space':
      # Constructing the SPARQL query
        query = f""" SELECT distinct ?item ?QID ?itemLabel ?itemDescription
        WHERE{{
          ?item rdfs:label ?itemLabel.
          FILTER(REGEX(?itemLabel, ?name )).
          VALUES ?name {{"^{input_query} "@en}} .
          FILTER(LANG(?itemLabel)="en").
          ?item wdt:P31 ?type .
          VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                                wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                                wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                                wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities

          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.
        #the next 3 lines will give the QID in the column QID
        #This query will bind the substring of ?uri starting from the 32nd character to a new variable ?code
        BIND (STR (?item) AS ?uri) .
        FILTER (STRSTARTS (?uri, "http://www.wikidata.org/entity/Q")) .
        BIND (SUBSTR (?uri, 32) AS ?QID) .

        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """
    elif search_option == 'space_comma':
      # Constructing the SPARQL query
        query = f""" SELECT distinct ?item ?QID ?itemLabel ?itemDescription
        WHERE{{
          ?item rdfs:label ?itemLabel.
          FILTER(REGEX(?itemLabel, ?name )).
          VALUES ?name {{"^{input_query}, "@en}} .
          FILTER(LANG(?itemLabel)="en").
          ?item wdt:P31 ?type .
          VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                                wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                                wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                                wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities

          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.
        #the next 3 lines will give the QID in the column QID
        #This query will bind the substring of ?uri starting from the 32nd character to a new variable ?code
        BIND (STR (?item) AS ?uri) .
        FILTER (STRSTARTS (?uri, "http://www.wikidata.org/entity/Q")) .
        BIND (SUBSTR (?uri, 32) AS ?QID) .

        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """

    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()
    output = data["results"]["bindings"]
    return output

In [ ]:
# Importing the modules
import requests
import random
import pandas as pd
import numpy as np


# Defining a function that takes an input from user and returns the Wikidata ID and label
def choose_company(input_name, search_option='all'):
    # remove any space at the beginning/end of the string
    input_name = input_name.strip()
    # Initializing an empty list to store the IDs and labels
    ids_and_labels = []
    # Make the query
    results = searchCompanyByName(input_name, search_option)
    if len(results) != 0:
        # Looping through the results and appending them to the list
        for result in results:
            # Getting the Wikidata ID and label
            wikidata_id = result["item"]["value"].split("/")[-1]
            wikidata_label = result["itemLabel"]["value"]
            # Appending a tuple of ID and label to the list
            ids_and_labels.append((wikidata_id, wikidata_label))
    return ids_and_labels


### user calls the function

In [ ]:
t = choose_company('Tesla')
t

[('Q106868050', 'Tesla Energy'),
 ('Q105162434', 'Tesla India'),
 ('Q22674561', 'Tesla Grohmann Automation'),
 ('Q1548225', 'Tesla'),
 ('Q116876087', 'Tesla (Thailand)'),
 ('Q4920135', 'Tesla Electric Light & Manufacturing'),
 ('Q478214', 'Tesla, Inc.'),
 ('Q27150172', 'Tesla US dealership disputes')]

In [ ]:
t[6]

('Q478214', 'Tesla, Inc.')

In [ ]:
s = choose_company('Tesla, Inc.', 'exact')
s

[('Q478214', 'Tesla, Inc.')]

In [ ]:
w = choose_company('Polestar')
w

[('Q4047174', 'Polestar')]

In [ ]:
z = [s[0],w[0]]
z

[('Q478214', 'Tesla, Inc.'), ('Q4047174', 'Polestar')]

In [ ]:
y = [t[6],w[0]]
y

[('Q478214', 'Tesla, Inc.'), ('Q4047174', 'Polestar')]

In [ ]:
QIDs = [x[0] for x in z]
print(QIDs)

['Q478214', 'Q4047174']


## FUNCTION 2. get_companies_network(QIDs, num_runs=5,5,5,5)

### Function to get parent info

In [ ]:
# Checking if entity has a parent, and if so how many
def has_parent(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT ?parent (IF (BOUND(?parent), COUNT(DISTINCT ?parent), 0) AS ?count)
          WHERE {{
                VALUES ?item {{wd:{items}}}.
                ?item wdt:P31 ?type .
                VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                            wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                            wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                            wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities
                ?article schema:about ?item .
                ?article schema:inLanguage "en" .
                ?article schema:isPartOf <https://en.wikipedia.org/>.

                OPTIONAL {{ ?item wdt:P749 ?parent. }}

                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                GROUP BY ?parent
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()
    # Extracting all the results
    results = data["results"]["bindings"]
    # Check if results is empty
    if not results:
      # Return zero
      return 0
    else:
      # Get the count value
      counts = results[0]["count"]["value"]
      # Make counts integers
      counts = int(counts)
      # Return the count as an integer
      return counts





In [ ]:
# Defining a function that takes a Wikidata ID and returns a dictionary of information about its industry, parent organizations and subsidiaries
def get_company_parent_info(wikidata_id):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f"""
    SELECT DISTINCT ?item
                    (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                    ?itemLabel
                    ?itemcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?industryLabel; separator="; ") AS ?industries)
                    ?parent
                    (REPLACE(STR(?parent), "http://www.wikidata.org/entity/", "") AS ?pQID)
                    ?parentLabel
                    ?parentcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?pindustryLabel; separator="; ") AS ?pindustries)
                    ?proportion
                    ?proportionofLabel
                    ?pointoftime
                    ?starttime
                    ?endtime
    WHERE {{
      VALUES ?item {{wd:{wikidata_id}}}.
      ?item wdt:P31 ?type .
      ?article schema:about ?item .
      ?article schema:inLanguage "en" .
      ?article schema:isPartOf <https://en.wikipedia.org/>.

        OPTIONAL {{ ?item rdfs:label ?itemLabel. FILTER(LANG(?itemLabel) = "en") }}
        OPTIONAL {{ ?item     wdt:P17    ?itemcountry .
                    ?itemcountry  rdfs:label  ?itemcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?itemcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item     wdt:P452    ?industry .
                    ?industry  rdfs:label  ?industryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?industryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item wdt:P749 ?parent. }}
        OPTIONAL {{ ?parent     wdt:P452    ?pindustry .
                    ?pindustry  rdfs:label  ?pindustryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?pindustryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?parent     wdt:P17    ?parentcountry .
                    ?parentcountry  rdfs:label  ?parentcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?parentcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item  p:P749 [ps:P749 ?parent; pq:P1107 ?proportion] }}
        OPTIONAL {{
                  ?item  p:P749 [ps:P749 ?parent; pq:P642 ?proportionof] .
                  FILTER(?proportionof = wd:Q144368)
                 }}
        OPTIONAL {{ ?item p:P749 [ps:P749 ?parent; pq:P585 ?pointoftime] }}
        OPTIONAL {{
                  ?item p:P749 [ps:P749 ?parent; pq:P585 ?pointoftime] .
                  FILTER(BOUND(?pointoftime) && DATATYPE(?pointoftime) = xsd:dateTime).
                  # get the latest record first
                  BIND(NOW() - ?pointoftime AS ?distance).
                  FILTER (MIN (?distance)) .
                 }}
        OPTIONAL {{ ?item  p:P749 [ps:P749 ?parent; pq:P580 ?starttime] }}
        OPTIONAL {{ ?item  p:P749 [ps:P749 ?parent; pq:P582 ?endtime] }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY  ?item ?QID ?itemLabel ?itemcountryLabel ?parent ?pQID ?parentLabel ?parentcountryLabel ?proportion ?proportionofLabel ?pointoftime ?starttime ?endtime
    ORDER BY ?parentLabel
    LIMIT 10000
    """

    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # defining url
    url = 'https://query.wikidata.org/sparql'
    # Sending the request to the Wikidata endpoint with the user agent header
    r = requests.get(url, params={'format': 'json', 'query': query}, headers={"User-Agent": user_agent})
    # extracts the results and bindings keys from the JSON response returned by the Wikidata SPARQL endpoint
    data = r.json()['results']['bindings']
    # converts the JSON data into a pandas DataFrame
    df = pd.json_normalize(data)
    #  checks if an element is a dictionary and if so, extracts the value key from it
    df = df.applymap(lambda x: x['value'] if isinstance(x, dict) else x)
    # selects only the columns that end with .value.
    df = df[[col for col in df.columns if col.endswith('.value')]]
    # removes the .value suffix from each column name
    df.columns = [col.split('.')[0] for col in df.columns]

    return df


### Function to get owned by info

In [ ]:
# Checking if entity has a ownedby (which is also a type of company), and if so how many
def has_ownedby(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT ?ownedby (IF (BOUND(?ownedby), COUNT(DISTINCT ?ownedby), 0) AS ?count)
          WHERE {{
                VALUES ?item {{wd:{items}}}.
                ?item wdt:P31 ?type .
                VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                            wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                            wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                            wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities
                ?article schema:about ?item .
                ?article schema:inLanguage "en" .
                ?article schema:isPartOf <https://en.wikipedia.org/>.

                OPTIONAL {{ ?item wdt:P127 ?ownedby. }}

                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                GROUP BY ?ownedby
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()
    # Extracting all the results
    results = data["results"]["bindings"]
    # Check if results is empty
    if not results:
      # Return zero
      return 0
    else:
      # Get the count value
      counts = results[0]["count"]["value"]
      # Make counts integers
      counts = int(counts)
      # Return the count as an integer
      return counts





In [ ]:
# Defining a function that takes a Wikidata ID and returns a dictionary of information about its industry, parent organizations and subsidiaries
def get_company_ownedby_info(wikidata_id):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f"""
    SELECT DISTINCT ?item
                    (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                    ?itemLabel
                    ?itemcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?industryLabel; separator="; ") AS ?industries)
                    ?ownedby
                    (REPLACE(STR(?ownedby), "http://www.wikidata.org/entity/", "") AS ?obQID)
                    ?ownedbyLabel
                    ?ownedbycountryLabel
                    (GROUP_CONCAT ( DISTINCT ?obindustryLabel; separator="; ") AS ?obindustries)
                    ?proportion
                    ?proportionofLabel
                    ?pointoftime
                    ?starttime
                    ?endtime
    WHERE {{
      VALUES ?item {{wd:{wikidata_id}}}.
      ?item wdt:P31 ?type .
      ?article schema:about ?item .
      ?article schema:inLanguage "en" .
      ?article schema:isPartOf <https://en.wikipedia.org/>.

        OPTIONAL {{ ?item rdfs:label ?itemLabel. FILTER(LANG(?itemLabel) = "en") }}
        OPTIONAL {{ ?item     wdt:P17    ?itemcountry .
                    ?itemcountry  rdfs:label  ?itemcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?itemcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item     wdt:P452    ?industry .
                    ?industry  rdfs:label  ?industryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?industryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item wdt:P127 ?ownedby. }}
        OPTIONAL {{ ?ownedby     wdt:P452    ?obindustry .
                    ?obindustry  rdfs:label  ?obindustryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?obindustryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?ownedby     wdt:P17    ?ownedbycountry .
                    ?ownedbycountry  rdfs:label  ?ownedbycountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?ownedbycountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item  p:P127 [ps:P127 ?ownedby; pq:P1107 ?proportion] }}
        OPTIONAL {{
                  ?item  p:P127 [ps:P127 ?ownedby; pq:P642 ?proportionof] .
                  FILTER(?proportionof = wd:Q144368)
                 }}
        OPTIONAL {{ ?item p:P127 [ps:P127 ?ownedby; pq:P585 ?pointoftime] }}
        OPTIONAL {{
                  ?item p:P127 [ps:P127 ?ownedby; pq:P585 ?pointoftime] .
                  FILTER(BOUND(?pointoftime) && DATATYPE(?pointoftime) = xsd:dateTime).
                  # get the latest record first
                  BIND(NOW() - ?pointoftime AS ?distance).
                  FILTER (MIN (?distance)) .
                 }}
        OPTIONAL {{ ?item  p:P127 [ps:P127 ?ownedby; pq:P580 ?starttime] }}
        OPTIONAL {{ ?item  p:P127 [ps:P127 ?ownedby; pq:P582 ?endtime] }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY  ?item ?QID ?itemLabel ?itemcountryLabel ?ownedby ?obQID ?ownedbyLabel ?ownedbycountryLabel ?proportion ?proportionofLabel ?pointoftime ?starttime ?endtime
    ORDER BY ?ownedbyLabel
    LIMIT 10000
    """

     # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # defining url
    url = 'https://query.wikidata.org/sparql'
    # Sending the request to the Wikidata endpoint with the user agent header
    r = requests.get(url, params={'format': 'json', 'query': query}, headers={"User-Agent": user_agent})
    # extracts the results and bindings keys from the JSON response returned by the Wikidata SPARQL endpoint
    data = r.json()['results']['bindings']
    # converts the JSON data into a pandas DataFrame
    df = pd.json_normalize(data)
    #  checks if an element is a dictionary and if so, extracts the value key from it
    df = df.applymap(lambda x: x['value'] if isinstance(x, dict) else x)
    # selects only the columns that end with .value.
    df = df[[col for col in df.columns if col.endswith('.value')]]
    # removes the .value suffix from each column name
    df.columns = [col.split('.')[0] for col in df.columns]

    return df


### Function to get subsidiary info

In [ ]:
# Checking if entity has a subsidiary (which is also a type of company), and if so how many
def has_subsidiary(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT ?subsidiary (IF (BOUND(?subsidiary), COUNT(DISTINCT ?subsidiary), 0) AS ?count)
          WHERE {{
                VALUES ?item {{wd:{items}}}.
                ?item wdt:P31 ?type .
                VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                            wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                            wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                            wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities
                ?article schema:about ?item .
                ?article schema:inLanguage "en" .
                ?article schema:isPartOf <https://en.wikipedia.org/>.

                OPTIONAL {{ ?item wdt:P355 ?subsidiary. }}

                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                GROUP BY ?subsidiary
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()
    # Extracting all the results
    results = data["results"]["bindings"]
    # Check if results is empty
    if not results:
      # Return zero
      return 0
    else:
      # Get the count value
      counts = results[0]["count"]["value"]
      # Make counts integers
      counts = int(counts)
      # Return the count as an integer
      return counts





In [ ]:
# getting company's subsidaries
def get_company_subsidiary_info(wikidata_id):
    import requests
    import pandas as pd
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    query = f"""
        SELECT DISTINCT ?item
                        (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                        ?itemLabel
                        ?itemcountryLabel
                        (GROUP_CONCAT ( DISTINCT ?industryLabel; separator="; ") AS ?industries)
                        ?subsidiary
                        (REPLACE(STR(?subsidiary), "http://www.wikidata.org/entity/", "") AS ?sQID)
                        ?subsidiaryLabel
                        ?subsidiarycountryLabel
                        (GROUP_CONCAT ( DISTINCT ?sindustryLabel; separator="; ") AS ?sindustries)
                        ?proportion
                        ?proportionofLabel
                        ?pointoftime
                        ?starttime
                        ?endtime
        WHERE {{
          VALUES ?item {{wd:{wikidata_id}}}.
          ?item wdt:P31 ?type .
          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.

            OPTIONAL {{ ?item rdfs:label ?itemLabel. FILTER(LANG(?itemLabel) = "en") }}
            OPTIONAL {{ ?item     wdt:P17    ?itemcountry .
                    ?itemcountry  rdfs:label  ?itemcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?itemcountryLabel ), "en" ) )
                  }}
            OPTIONAL {{ ?item     wdt:P452    ?industry .
                        ?industry  rdfs:label  ?industryLabel
                        FILTER ( LANGMATCHES ( LANG ( ?industryLabel ), "en" ) )
                      }}
            OPTIONAL {{ ?item wdt:P355 ?subsidiary. }}
            OPTIONAL {{ ?subsidiary     wdt:P452    ?sindustry .
                        ?sindustry  rdfs:label  ?sindustryLabel
                        FILTER ( LANGMATCHES ( LANG ( ?sindustryLabel ), "en" ) )
                      }}
            OPTIONAL {{ ?subsidiary     wdt:P17    ?subsidiarycountry .
                    ?subsidiarycountry  rdfs:label  ?subsidiarycountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?subsidiarycountryLabel ), "en" ) )
                  }}
            OPTIONAL {{ ?item  p:P355 [ps:P355 ?subsidiary; pq:P1107 ?proportion] }}
            OPTIONAL {{
                        ?item  p:P355 [ps:P355 ?subsidiary; pq:P642 ?proportionof] .
                        FILTER(?proportionof = wd:Q144368)
                     }}
            OPTIONAL {{ ?item p:P355 [ps:P355 ?subsidiary; pq:P585 ?pointoftime] }}
            OPTIONAL {{
                        ?item p:P355 [ps:P355 ?subsidiary; pq:P585 ?pointoftime] .
                        FILTER(BOUND(?pointoftime) && DATATYPE(?pointoftime) = xsd:dateTime).
                        # get the latest record first
                        BIND(NOW() - ?pointoftime AS ?distance).
                        FILTER (MIN (?distance)) .
                      }}
            OPTIONAL {{ ?item  p:P355 [ps:P355 ?subsidiary; pq:P580 ?starttime] }}
            OPTIONAL {{ ?item  p:P355 [ps:P355 ?subsidiary; pq:P582 ?endtime] }}
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        GROUP BY  ?item ?QID ?itemLabel ?itemcountryLabel ?subsidiary ?sQID ?subsidiaryLabel ?subsidiarycountryLabel ?proportion ?proportionofLabel ?pointoftime ?starttime ?endtime
        ORDER BY ?subsidiaryLabel
        LIMIT 10000
        """

    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # defining url
    url = 'https://query.wikidata.org/sparql'
    # Sending the request to the Wikidata endpoint with the user agent header
    r = requests.get(url, params={'format': 'json', 'query': query}, headers={"User-Agent": user_agent})
    # extracts the results and bindings keys from the JSON response returned by the Wikidata SPARQL endpoint
    data = r.json()['results']['bindings']
    # converts the JSON data into a pandas DataFrame
    df = pd.json_normalize(data)
    #  checks if an element is a dictionary and if so, extracts the value key from it
    df = df.applymap(lambda x: x['value'] if isinstance(x, dict) else x)
    # selects only the columns that end with .value.
    df = df[[col for col in df.columns if col.endswith('.value')]]
    # removes the .value suffix from each column name
    df.columns = [col.split('.')[0] for col in df.columns]

    return df

### Function to get owner of info

In [ ]:
# Checking if entity has a ownerof (which is also a type of company), and if so how many
def has_ownerof(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT ?ownerof (IF (BOUND(?ownerof), COUNT(DISTINCT ?ownerof), 0) AS ?count)
          WHERE {{
                VALUES ?item {{wd:{items}}}.
                ?item wdt:P31 ?type .
                VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                            wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                            wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                            wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities
                ?article schema:about ?item .
                ?article schema:inLanguage "en" .
                ?article schema:isPartOf <https://en.wikipedia.org/>.

                OPTIONAL {{ ?item wdt:P1830 ?ownerof. }}

                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                GROUP BY ?ownerof
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()
    # Extracting all the results
    results = data["results"]["bindings"]
    # Check if results is empty
    if not results:
      # Return zero
      return 0
    else:
      # Get the count value
      counts = results[0]["count"]["value"]
      # Make counts integers
      counts = int(counts)
      # Return the count as an integer
      return counts





In [ ]:
# Defining a function that takes a Wikidata ID and returns a dictionary of information about its industry, parent organizations and subsidiaries
def get_company_ownerof_info(wikidata_id):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f"""
    SELECT DISTINCT ?item
                    (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                    ?itemLabel
                    ?itemcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?industryLabel; separator="; ") AS ?industries)
                    ?ownerof
                    (REPLACE(STR(?ownerof), "http://www.wikidata.org/entity/", "") AS ?ooQID)
                    ?ownerofLabel
                    ?ownerofcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?ooindustryLabel; separator="; ") AS ?ooindustries)
                    ?proportion
                    ?proportionofLabel
                    ?pointoftime
                    ?starttime
                    ?endtime
    WHERE {{
      VALUES ?item {{wd:{wikidata_id}}}.
      ?item wdt:P31 ?type .
      ?article schema:about ?item .
      ?article schema:inLanguage "en" .
      ?article schema:isPartOf <https://en.wikipedia.org/>.

        OPTIONAL {{ ?item rdfs:label ?companyLabel. FILTER(LANG(?companyLabel) = "en") }}
        OPTIONAL {{ ?item     wdt:P17    ?itemcountry .
                    ?itemcountry  rdfs:label  ?itemcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?itemcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item     wdt:P452    ?industry .
                    ?industry  rdfs:label  ?industryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?industryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item wdt:P1830 ?ownerof. }}
        OPTIONAL {{ ?ownerof     wdt:P452    ?ooindustry .
                    ?ooindustry  rdfs:label  ?ooindustryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?ooindustryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?ownerof     wdt:P17    ?ownerofcountry .
                    ?ownerofcountry  rdfs:label  ?ownerofcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?ownerofcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item  p:P1830 [ps:P1830 ?ownerof; pq:P1107 ?proportion] }}
        OPTIONAL {{
                    ?item  p:P1830 [ps:P1830 ?ownerof; pq:P642 ?proportionof] .
                    FILTER(?proportionof = wd:Q144368)
                  }}
        OPTIONAL {{ ?item p:P1830 [ps:P1830 ?ownerof; pq:P585 ?pointoftime] }}
        OPTIONAL {{
                    ?item p:P1830 [ps:P1830 ?ownerof; pq:P585 ?pointoftime] .
                    FILTER(BOUND(?pointoftime) && DATATYPE(?pointoftime) = xsd:dateTime).
                    # get the latest record first
                    BIND(NOW() - ?pointoftime AS ?distance).
                    FILTER (MIN (?distance)) .
                  }}
        OPTIONAL {{ ?item  p:P1830 [ps:P1830 ?ownerof; pq:P580 ?starttime] }}
        OPTIONAL {{ ?item  p:P1830 [ps:P1830 ?ownerof; pq:P582 ?endtime] }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY  ?item ?QID ?itemLabel ?itemcountryLabel ?ownerof ?ooQID ?ownerofLabel ?ownerofcountryLabel ?proportion ?proportionofLabel ?pointoftime ?starttime ?endtime
    ORDER BY ?ownerofLabel
    LIMIT 10000
    """

    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # defining url
    url = 'https://query.wikidata.org/sparql'
    # Sending the request to the Wikidata endpoint with the user agent header
    r = requests.get(url, params={'format': 'json', 'query': query}, headers={"User-Agent": user_agent})
    # extracts the results and bindings keys from the JSON response returned by the Wikidata SPARQL endpoint
    data = r.json()['results']['bindings']
    # converts the JSON data into a pandas DataFrame
    df = pd.json_normalize(data)
    #  checks if an element is a dictionary and if so, extracts the value key from it
    df = df.applymap(lambda x: x['value'] if isinstance(x, dict) else x)
    # selects only the columns that end with .value.
    df = df[[col for col in df.columns if col.endswith('.value')]]
    # removes the .value suffix from each column name
    df.columns = [col.split('.')[0] for col in df.columns]

    return df

### Define the main function

In [ ]:
def tidy_each_df(df):
  #split companies' industries string and return them as list, then drop duplicates, and finally return as string again
  df.iloc[:, 4] = df.iloc[:,4].str.split("; ").apply(lambda x: "; ".join(set(word.lower() for word in x)))
  # drop parents' industries duplicates, then return as string again
  df.iloc[:, 9] = df.iloc[:, 9].str.split("; ").apply(lambda x: "; ".join(set(word.lower() for word in x)))
  # check if all proprtions are between 0 and 1, if above 1, then devide by 100, otherwise leave it
  if 'proportion' in df.columns:
    df['proportion'] = df['proportion'].astype(float)
    df['proportion'] = np.where(df['proportion'].isnull(), df['proportion'],
                                np.where(df['proportion'] > 1, df['proportion']/100,
                                         df['proportion']))
  df = df.drop_duplicates().reset_index(drop=True)
  return df

In [ ]:
# companies_network function

# define the main function
def get_companies_network(ids, levels=(5,5,5,5)):
  import numpy as np
  import pandas as pd
  # Create an empty dataframe for parent
  p_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'parent', \
                               'pQID', 'parentLabel', 'parentcountryLabel', 'pindustries', 'proportion', \
                               'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  # Create an empty dataframe for owned by
  ob_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'ownedby', \
                                'obQID', 'ownedbyLabel', 'ownedbycountryLabel', 'obindustries', 'proportion', \
                                'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  # Create an empty dataframe for subsidiary
  s_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'subsidiary', \
                               'sQID', 'subsidiaryLabel', 'subsidiarycountryLabel', 'sindustries', 'proportion', \
                               'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  # Create an empty dataframe for owner of
  oo_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'ownerof', \
                                'ooQID', 'ownerofLabel', 'ownerofcountryLabel', 'ooindustries', 'proportion', \
                                'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])

  # initialize a list to store the results of each run
  results = {"p_df": [], "ob_df": [], "s_df": [], "oo_df": []}
  # making sure only those QIDs in root_QIDs that starts with "Q"
  root_QIDs = [item for item in ids if item.startswith("Q")]
  # Find the maximum number of levels
  max_levels = max(levels)
  # Iterate from 0 to max_levels-1
  for j in range(max_levels):
    # Iterate over each element of the levels tuple
    for i in range(len(levels)):
      # Check if j is less than the value of levels[i]
      if j < levels[i]:
          # Iterate over each element of the item list
          if i == 0:
            # making sure only those QIDs in root_QIDs that starts with "Q"
            root_QIDs = [item for item in root_QIDs if item.startswith("Q")]
            ## HAS PARENT
            # make a new parent QID list with only those that has parent
            new_p_QIDs = []
            for id in root_QIDs:
              parent_count = has_parent(id)
              if parent_count > 0:
                new_p_QIDs.append(id)
            ## PARENT INFO
            # Check if new_p_QIDs is empty
            # get the parent info from the new QID list and concat it with the p_df dataframe
            for id in new_p_QIDs:
              p_df = pd.concat([p_df, get_company_parent_info(id)], ignore_index=True)
            # tidy up industries and proportions with function tidy_each_df()
            p_df = tidy_each_df(p_df)
          elif i == 1:
            # making sure only those QIDs in root_QIDs that starts with "Q"
            root_QIDs = [item for item in root_QIDs if item.startswith("Q")]
            ## HAS OWNED BY
            # make a new ownedby QID list with only those companies that has ownedby
            new_ob_QIDs = []
            for id in root_QIDs:
              ownedby_count = has_ownedby(id)
              if ownedby_count > 0:
                new_ob_QIDs.append(id)
            ## GET OWNED BY INFO
            for id in new_ob_QIDs:
              ob_df = pd.concat([ob_df, get_company_ownedby_info(id)], ignore_index=True)
            # tidy up industries and proportions with function tidy_each_df()
            ob_df = tidy_each_df(ob_df)
          elif i == 2:
            # making sure only those QIDs in root_QIDs that starts with "Q"
            root_QIDs = [item for item in root_QIDs if item.startswith("Q")]
            ## HAS SUBSIDIARY
            # make a new subsidiary QID list with only those companies that has subsidiary
            new_s_QIDs = []
            for id in root_QIDs:
              subsidiary_count = has_subsidiary(id)
              if subsidiary_count > 0:
                new_s_QIDs.append(id)
            ## GET SUBSIDIARY INFO
            # get the subsidiaries info from the new QID list and concat it with the s_df dataframe
            for id in new_s_QIDs:
              s_df = pd.concat([s_df, get_company_subsidiary_info(id)], ignore_index=True)
            # tidy up industries and proportions with function tidy_each_df()
            s_df = tidy_each_df(s_df)
          elif i == 3:
            # making sure only those QIDs in root_QIDs that starts with "Q"
            root_QIDs = [item for item in root_QIDs if item.startswith("Q")]
            ## HAS OWNER OF
            # make a new ownerof QID list with only those companies that has owner of
            new_oo_QIDs = []
            for id in root_QIDs:
              ownerof_count = has_ownerof(id)
              if ownerof_count > 0:
                new_oo_QIDs.append(id)
            ## GET OWNER OF INFO
            # get the owner of info from the new QID list and concat it with the oo_df dataframe
            for id in new_oo_QIDs:
              oo_df = pd.concat([oo_df, get_company_ownerof_info(id)], ignore_index=True)
            # tidy up industries and proportions with function tidy_each_df()
            oo_df = tidy_each_df(oo_df)
    # append results after each number of runs
    results['p_df'].append(p_df)
    results['ob_df'].append(ob_df)
    results['s_df'].append(s_df)
    results['oo_df'].append(oo_df)
    ## JOIN NEW UNIQUE QIDS
    #getting all the QIDs that will needs to be queried again
    # Store dataframes in a list
    df_list = [p_df, ob_df, s_df, oo_df]
    # Store column names in a list
    col_list = ['pQID', 'obQID', 'sQID', 'ooQID']
    # Create an empty list to store the column values
    val_list = []
    # Loop over the dataframes and the column names and append the column values to the list
    for df, col in zip(df_list, col_list):
      val_list.append(df[col].to_list())
    #collect the unique values from val_list
    new_list = []
    [new_list.append(x) for x in val_list if x not in new_list]
    # Convert the list to a pandas series
    original_series = pd.Series(new_list)
    # Use the explode method to split the nested lists into separate rows
    exploded_series = original_series.explode()
    # Use the drop_duplicates method to remove any duplicate values
    unique_series = exploded_series.drop_duplicates()
    # Convert the series back to a list
    new_QIDs = unique_series.tolist()
    #clean new_QIDs from 'nan' string values
    new_QIDs = [x for x in new_QIDs if str(x) != 'nan']
    # get a list of the new unique QIDs
    new1_QIDs = list(set(new_QIDs) - set(root_QIDs))
    # keep only those QIDs that starts with "Q"
    new2_QIDs = [item for item in new1_QIDs if item.startswith("Q")]
    #rename the new list of QIDs as root_QIDs
    root_QIDs = new2_QIDs

  # return the list of results
  return results

### user call function 2

In [ ]:
QIDs

['Q478214', 'Q4047174']

In [ ]:
# to get the companies network, it needs the QIDs as the first argument, \
# and for 2nd the number of levels to run can be specified for each relations \
# such as (parent / owned by / subsidiary / owner of) where, \
# the first two numbers are upwards relations and the last two are downwards.
test = get_companies_network(QIDs, (5,5,5,5))

In [ ]:
# the output is a python dictionary of dataframes, which are named \
# p_df for parent company query results \
# ob_df stands for owned by \
# s_df is for subsidiary \
# oo_df is for the owner of results.
test.keys()

dict_keys(['p_df', 'ob_df', 's_df', 'oo_df'])

In [ ]:
print(len(test)) # 4 dataframes: p_df, ob_df, s_df, oo_df
print(len(test['p_df'])) # no of times the dataset was appended
print(len(test['p_df'][-1])) # results of the last run

4
5
41


In [ ]:
print(len(test)) # 4 dataframes: p_df, ob_df, s_df, oo_df
print(len(test['ob_df'])) # no of times the dataset was appended
print(len(test['ob_df'][-1])) # results of the last run

4
5
62


In [ ]:
print(len(test)) # 4 dataframes: p_df, ob_df, s_df, oo_df
print(len(test['s_df'])) # no of times the dataset was appended
print(len(test['s_df'][-1])) # results of the last run

4
5
97


In [ ]:
print(len(test)) # 4 dataframes: p_df, ob_df, s_df, oo_df
print(len(test['oo_df'])) # no of times the dataset was appended
print(len(test['oo_df'][-1])) # results of the last run

4
5
72


In [ ]:
p_data = test['p_df'][-1]
p_data.tail()

,item,QID,itemLabel,itemcountryLabel,industries,parent,pQID,parentLabel,parentcountryLabel,pindustries,proportion,proportionofLabel,pointoftime,starttime,endtime
36,http://www.wikidata.org/entity/Q1201531,Q1201531,Detroit Diesel,United States of America,automotive industry,http://www.wikidata.org/entity/Q16834431,Q16834431,Daimler Truck North America,United States of America,automotive industry,NaN,NaN,NaN,NaN,NaN
37,http://www.wikidata.org/entity/Q2378642,Q2378642,Thomas Built Buses,NaN,automotive industry,http://www.wikidata.org/entity/Q16834431,Q16834431,Daimler Truck North America,United States of America,automotive industry,NaN,NaN,NaN,NaN,NaN
38,http://www.wikidata.org/entity/Q1107991,Q1107991,BharatBenz,India,automotive industry,http://www.wikidata.org/entity/Q5209438,Q5209438,Daimler India Commercial Vehicles Pvt. Ltd.,NaN,automotive industry,NaN,NaN,NaN,NaN,NaN
39,http://www.wikidata.org/entity/Q386598,Q386598,EvoBus,Germany,automotive industry,http://www.wikidata.org/entity/Q1157624,Q1157624,Daimler Truck,Germany,automotive industry,NaN,NaN,NaN,NaN,NaN
40,http://www.wikidata.org/entity/Q5209438,Q5209438,Daimler India Commercial Vehicles Pvt. Ltd.,NaN,automotive industry,http://www.wikidata.org/entity/Q1157624,Q1157624,Daimler Truck,Germany,automotive industry,NaN,NaN,NaN,NaN,NaN


In [ ]:
p_data2 = p_data.drop_duplicates()
p_data2.tail()

,item,QID,itemLabel,itemcountryLabel,industries,parent,pQID,parentLabel,parentcountryLabel,pindustries,proportion,proportionofLabel,pointoftime,starttime,endtime
36,http://www.wikidata.org/entity/Q1201531,Q1201531,Detroit Diesel,United States of America,automotive industry,http://www.wikidata.org/entity/Q16834431,Q16834431,Daimler Truck North America,United States of America,automotive industry,NaN,NaN,NaN,NaN,NaN
37,http://www.wikidata.org/entity/Q2378642,Q2378642,Thomas Built Buses,NaN,automotive industry,http://www.wikidata.org/entity/Q16834431,Q16834431,Daimler Truck North America,United States of America,automotive industry,NaN,NaN,NaN,NaN,NaN
38,http://www.wikidata.org/entity/Q1107991,Q1107991,BharatBenz,India,automotive industry,http://www.wikidata.org/entity/Q5209438,Q5209438,Daimler India Commercial Vehicles Pvt. Ltd.,NaN,automotive industry,NaN,NaN,NaN,NaN,NaN
39,http://www.wikidata.org/entity/Q386598,Q386598,EvoBus,Germany,automotive industry,http://www.wikidata.org/entity/Q1157624,Q1157624,Daimler Truck,Germany,automotive industry,NaN,NaN,NaN,NaN,NaN
40,http://www.wikidata.org/entity/Q5209438,Q5209438,Daimler India Commercial Vehicles Pvt. Ltd.,NaN,automotive industry,http://www.wikidata.org/entity/Q1157624,Q1157624,Daimler Truck,Germany,automotive industry,NaN,NaN,NaN,NaN,NaN


In [ ]:
s_data = test['s_df'][-1]
s_data.tail()

,item,QID,itemLabel,itemcountryLabel,industries,subsidiary,sQID,subsidiaryLabel,subsidiarycountryLabel,sindustries,proportion,proportionofLabel,pointoftime,starttime,endtime
93,http://www.wikidata.org/entity/Q833456,Q833456,Rolls-Royce Power Systems,Germany,mechanical engineering,http://www.wikidata.org/entity/Q1880663,Q1880663,Woodward L'Orange GmbH,Germany,mechanical engineering,NaN,NaN,NaN,NaN,NaN
94,http://www.wikidata.org/entity/Q181114,Q181114,Stellantis North America,United States of America,automotive industry,http://www.wikidata.org/entity/Q4807118,Q4807118,Askam,Turkey,automotive industry,NaN,NaN,NaN,1962-01-01T00:00:00Z,1978-01-01T00:00:00Z
95,http://www.wikidata.org/entity/Q181114,Q181114,Stellantis North America,United States of America,automotive industry,http://www.wikidata.org/entity/Q5767794,Q5767794,Chrysler Fevre Argentina S.A.,Argentina,automotive industry,NaN,NaN,NaN,1959-01-01T00:00:00Z,1979-01-01T00:00:00Z
96,http://www.wikidata.org/entity/Q181114,Q181114,Stellantis North America,United States of America,automotive industry,http://www.wikidata.org/entity/Q1789249,Q1789249,Street & Racing Technology,United States of America,automotive industry,NaN,NaN,NaN,NaN,NaN
97,http://www.wikidata.org/entity/Q30378,Q30378,Kamaz,Russia,automotive industry,http://www.wikidata.org/entity/Q16699372,Q16699372,ZMA,Russia,automotive industry,NaN,NaN,NaN,NaN,NaN


In [ ]:
test2 = get_companies_network(QIDs)


In [ ]:
print(len(test2)) # 4 dataframes: p_df, ob_df, s_df, oo_df
print(len(test2['oo_df'])) # no of times the dataset was appended
print(len(test2['oo_df'][-1])) # results of the last run

4
5
71


## FUNCTION 3. visualise_b2b_network()

### Prepare dataset: including various functions such as is_human(), get_description() etc

In [ ]:
# Checking if entity is a human, then get description too
def is_human(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT DISTINCT
                                ?human
                                (REPLACE(STR(?human), "http://www.wikidata.org/entity/", "") AS ?QID)
                                ?humanLabel
                                ?humanDescription
      WHERE {{
            VALUES ?human {{wd:{items}}} .
            ?human wdt:P31/wdt:P279* wd:Q5 .
            ?article schema:about ?human .
            ?article schema:inLanguage "en" .
            ?article schema:isPartOf <https://en.wikipedia.org/>.
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            FILTER (LANG (?humanDescription) = "en")
            }}
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()

    # Initializing an empty dictionary to store the information
    info = {}
    # Looping through the results
    for result in data["results"]["bindings"]:
        # Adding the company's QID to the dictionary if exists
        if "QID" in result:
            info["QID"] = result["QID"]["value"]
        # Adding the company label to the dictionary if exists
        if "humanLabel" in result:
            info["name"] = result["humanLabel"]["value"]
        # Adding the industry label to the dictionary if exists
        if "humanDescription" in result:
            info["descr"] = result["humanDescription"]["value"]
    # Returning the dictionary of information
    return info



In [ ]:
# Getting description for the unknown items
def get_description(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT DISTINCT
                                ?item
                                (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                                ?itemLabel
                                ?itemDescription
      WHERE {{ wd:{items} rdfs:label ?itemLabel .
            OPTIONAL {{ wd:{items} schema:description ?itemDescription }} .
            BIND (wd:{items} AS ?item) .
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            FILTER (LANG (?itemDescription) = "en")
            }}
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()

    # Initializing an empty dictionary to store the information
    info = {}
    # Looping through the results
    for result in data["results"]["bindings"]:
        # Adding the company's QID to the dictionary if exists
        if "QID" in result:
            info["QID"] = result["QID"]["value"]
        # Adding the company label to the dictionary if exists
        if "itemLabel" in result:
            info["name"] = result["itemLabel"]["value"]
        # Adding the industry label to the dictionary if exists
        if "itemDescription" in result:
            info["descr"] = result["itemDescription"]["value"]
    # Returning the dictionary of information
    return info



In [ ]:
def clear_each_df(df):
  import pandas as pd
  # convert endtime column to string and filter out rows with valid dates
  if 'endtime' in df.columns:
    df['endtime'] = df['endtime'].astype(str)
    df1 = df[~df['endtime'].str.contains('\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z')]
  new_df =pd.DataFrame(columns=df.columns)
  # create a set of unique QID and obQID pairs
  qid1_qid2_pairs = set(zip(df1.iloc[:, 1], df1.iloc[:, 6]))
  # loop over the unique pairs and select the last row for each pair
  for qid1, qid2 in qid1_qid2_pairs:
      temp_df = df1.loc[(df1.iloc[:, 1] == qid1) & (df1.iloc[:, 6] == qid2)]
      temp_df = temp_df.drop_duplicates().reset_index(drop = True)
      if len(temp_df) == 1:
          new_row = temp_df.iloc[0]
          new_df = pd.concat([new_df, new_row.to_frame().T])
      if len(temp_df) >= 2:
          # check if proportionofLabel column is in temp_df
          if 'proportionofLabel' in temp_df:
              # check if proportionofLabel column has any value
              if any(temp_df['proportionofLabel']):
                  # use list comprehension to filter rows with proportionofLabel equal to 'authorised capital'
                  auth_cap_rows = [row for row in temp_df.itertuples() if row.proportionofLabel == 'authorised capital']
                  if auth_cap_rows:
                      # select the last row from the filtered list
                      new_row = auth_cap_rows[-1]
                      new_df = pd.concat([new_df, pd.Series(new_row._asdict()).to_frame().T])
          else:
              # return auth_cap_rows empty
              auth_cap_rows = []
          # use list comprehension to filter rows with pointoftime or starttime containing valid dates
          date_rows = [row for row in temp_df.itertuples() if '\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z' in (row.pointoftime or row.starttime)]
          if date_rows:
              # use max function to find the latest date from the filtered list
              latest_date = max(date_rows, key=lambda x: x.pointoftime or x.starttime)
              new_row = latest_date
              new_df = pd.concat([new_df, pd.Series(new_row._asdict()).to_frame().T])
  # reset index and drop duplicates
  new_df.reset_index(drop=True, inplace=True)
  new_df = new_df.drop_duplicates().reset_index(drop = True)

  # add those relations that has an endtime to the new dataframes
  if 'endtime' in df.columns:
    df['endtime'] = df['endtime'].astype(str)
    df2 = df[df['endtime'].str.contains('\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z')]
    df2.loc[:, 'proportion'] = 4
    df2['endtime'] = df2['endtime'].str[:10]
    df2 = df2.drop_duplicates()
  new_df = pd.concat([new_df, df2]).reset_index(drop = True)

  return new_df


In [ ]:
def drop_dupl_joined(new_df2):
  ### remove the duplicates from the joined datasets
  # Step 1. save the single parent-child pairs in the new3_df2
  #         if there are 2 or more of the same parent-child pair, then \
  #         drop the ones within these that has blank p_proportion.
  # Step 2. in a new list comprehension (temp_step2_df) save the ones that now became single parent-child.
  #         if there are 2 or more and if one of the p_prop value is == 2, then \
  ##        drop this row with value == 2
  #         Finally, if  there are 2 or more and their p_prop value is equal, then keep only the last
  ##        (this will allow more values to pass through if they differ)
  # STEP 1
  # Use a set to store the unique cQID values
  cQID_set = set(new_df2['cQID'])
  # Use a generator expression to iterate over the pQID values
  pQID_gen = (i for i in new_df2['pQID'])
  # Use a list comprehension to filter the rows based on pQID and cQID values
  temp_new3_df2 = [new_df2.loc[(new_df2['pQID'] == i) & (new_df2['cQID'] == j)]
                  for i in pQID_gen for j in cQID_set]
  # Use another list comprehension to process the filtered rows based on the
  # length and p_proportion values
  new3_df2 = pd.concat([row.iloc[0].to_frame().T if len(row) == 1
                        else row.dropna(subset=['p_proportion']).loc[row['p_proportion'].notnull()]
                        for row in temp_new3_df2])
  new3_df2 = new3_df2.drop_duplicates().reset_index(drop = True)
  # STEP 2
  # Use a set to store the unique cQID values
  cQID_set2 = set(new3_df2['cQID'])
  # Use a generator expression to iterate over the pQID values
  pQID_gen2 = (i for i in new3_df2['pQID'])
  # Use a list comprehension to filter the rows based on pQID and cQID values
  temp_step2_df = [new3_df2.loc[(new3_df2['pQID'] == i) & (new3_df2['cQID'] == j)]
                  for i in pQID_gen2 for j in cQID_set2]
  # Drop the rows with p_proportion value equal to 2
  final_df = pd.concat([row.iloc[0].to_frame().T if len(row) == 1 \
                        else row.loc[row['p_proportion'] != 2] for row in temp_step2_df])
  # Keep only the last row if there are multiple rows with the same pQID and cQID
  # values and equal p_proportion values
  final_df = final_df.drop_duplicates(subset=['pQID', 'cQID', 'p_proportion'], keep='last')
  final_df = final_df.drop_duplicates().reset_index(drop = True)
  # fill the blank p_proportion values with number 3, this will mean unknown on the graph
  final_df['p_proportion'] = final_df['p_proportion'].fillna(3)
  return final_df

In [ ]:
def top_industries(final_df):
  import numpy as np

  # create a dictionary of industry frequencies
  # split industry columns by ";" and create separate columns
  df_split1 = final_df["p_industries"].str.split(";", expand=True)
  df_split2 = final_df["c_industries"].str.split(";", expand=True)
  # join the two dataframes horizontally
  df_split = pd.concat([df_split1, df_split2], axis=1)
  # reshape the dataframe so that each expression is in a single column
  df_melt = df_split.melt(value_name="expression")
  # drop the variable column
  df_melt = df_melt.drop("variable", axis=1)
  # count the frequency of each expression and convert to dictionary
  freq = df_melt["expression"].value_counts().to_dict()

  # define a custom function to pick the expression with highest frequency
  def pick_expression(lst):
    import numpy as np
    # split the list by semicolon
    lst = lst.split(";")
    # initialize the best expression and frequency
    best_exp = None
    best_freq = 0
    # loop through the list
    for exp in lst:
        # get the frequency of the expression
        freq_exp = freq.get(exp, 0)
        # if the frequency is higher than the best frequency, update the best expression and frequency
        if freq_exp > best_freq:
            best_exp = exp
            best_freq = freq_exp
    # return the best expression
    return best_exp

  # apply the custom function to each cell in col2
  final_df["top_p_industries"] = final_df["p_industries"].apply(pick_expression)
  final_df["top_c_industries"] = final_df["c_industries"].apply(pick_expression)

  # replace the empty strings with NaN
  final_df["top_p_industries"] = final_df["top_p_industries"].replace("", np.nan)
  final_df["top_c_industries"] = final_df["top_c_industries"].replace("", np.nan)
  final_df["p_industries"] = final_df["p_industries"].replace("", np.nan)
  final_df["c_industries"] = final_df["c_industries"].replace("", np.nan)

  # use isna method to get a boolean mask of missing values
  mask = final_df["top_p_industries"].isna()
  mask2 = final_df["top_c_industries"].isna()
  mask3 = final_df["p_industries"].isna()
  mask4 = final_df["c_industries"].isna()

  # use loc method to assign "unknown" to the rows where col2 is missing
  final_df.loc[mask, "top_p_industries"] = "unknown"
  final_df.loc[mask2, "top_c_industries"] = "unknown"
  final_df.loc[mask3, "p_industries"] = "unknown"
  final_df.loc[mask4, "c_industries"] = "unknown"

  final_df["top_p_industries"] = final_df["top_p_industries"].str.lstrip()
  final_df["top_c_industries"] = final_df["top_c_industries"].str.lstrip()

  # make a list of all those parent QIDs where the industry is "unknown"
  unique_values1 = final_df.loc[(final_df['top_p_industries'] == 'unknown'), 'pQID'].drop_duplicates().values.tolist()
  # making sure only those QIDs in unique_values that starts with "Q"
  unique_values1 = [item for item in unique_values1 if item.startswith("Q")]
  # create a dataframe for the results
  p_human_df1 = pd.DataFrame(columns=["QID", "name", "descr"])
  #initialising a new list to store the results
  results =[]
  # Loop through the values
  for id in unique_values1:
    # Run the module with the value as input and get the result
    result = pd.DataFrame(is_human(id), index=[0])
    # Appending the new dataframe to the list
    results.append(result)

  # join the new results to the dataframe
  p_human_df1 = pd.concat([p_human_df1] + results, axis=0, join='outer').drop_duplicates().reset_index(drop = True)

  #apply humans and their descriptions to new_df2 dataframe
  for index, row in final_df.iterrows():
      if row['pQID'] in p_human_df1['QID'].values:
          # get the index of the matching row in p_human_df1
          match_index = p_human_df1[p_human_df1['QID'] == row['pQID']].index
          # get the value from the "descr" column of p_human_df1
          descr_value = p_human_df1.loc[match_index[0], 'descr']
          # update the values in new_df2
          final_df.loc[index, 'p_industries'] = descr_value
          final_df.loc[index, 'top_p_industries'] = "human"

  # make a list of all those QIDs where the industry is "unknown"
  unique_values3 = final_df.loc[(final_df['top_p_industries'] == 'unknown'), 'pQID'].drop_duplicates().values.tolist()
  unique_values4 = final_df.loc[(final_df['top_c_industries'] == 'unknown'), 'cQID'].drop_duplicates().values.tolist()
  # making sure only those QIDs in unique_values that starts with "Q"
  unique_values3 = [item for item in unique_values3 if item.startswith("Q")]
  unique_values4 = [item for item in unique_values4 if item.startswith("Q")]
  # create a dataframe for the results
  p_item_df1 = pd.DataFrame(columns=["QID", "name", "descr"])
  #initialising a new list to store the results
  results =[]
  # Loop through the values
  for id in unique_values3:
    # Run the module with the value as input and get the result
    result = pd.DataFrame(get_description(id), index=[0])
    # Appending the new dataframe to the list
    results.append(result)

  # join the new results to the dataframe
  p_item_df1 = pd.concat([p_item_df1] + results, axis=0, join='outer').drop_duplicates().reset_index(drop = True)

  # create a dataframe for the results
  c_item_df2 = pd.DataFrame(columns=["QID", "name", "descr"])
  #initialising a new list to store the results
  results2 =[]
  # Loop through the values
  for id2 in unique_values4:
    # Run the module with the value as input and get the result
    result2 = pd.DataFrame(get_description(id2), index=[0])
    # Appending the new dataframe to the list
    results2.append(result2)

  # join the new results to the dataframe
  c_item_df2 = pd.concat([c_item_df2] + results2, axis=0, join='outer').drop_duplicates().reset_index(drop = True)

  #apply descriptions for unknown parents to new_df2 dataframe
  for index, row in final_df.iterrows():
      if row['pQID'] in p_item_df1['QID'].values:
          # get the index of the matching row in p_human_df1
          match_index = p_item_df1[p_item_df1['QID'] == row['pQID']].index
          # get the value from the "descr" column of p_human_df1
          descr_value = p_item_df1.loc[match_index[0], 'descr']
          # update the values in new_df2
          final_df.loc[index, 'p_industries'] = descr_value
          final_df.loc[index, 'top_p_industries'] = "other"

  #apply descriptions for unknown children to new_df2 dataframe
  for index, row in final_df.iterrows():
      if row['cQID'] in c_item_df2['QID'].values:
          # get the index of the matching row in p_human_df1
          match_index2 = c_item_df2[c_item_df2['QID'] == row['cQID']].index
          # get the value from the "descr" column of p_human_df1
          descr_value2 = c_item_df2.loc[match_index2[0], 'descr']
          # update the values in new_df2
          final_df.loc[index, 'c_industries'] = descr_value2
          final_df.loc[index, 'top_c_industries'] = "other"

  # convert the values in columns to strings
  final_df['p_industries'] = final_df['p_industries'].astype(str)
  final_df['top_p_industries'] = final_df['top_p_industries'].astype(str)
  final_df['c_industries'] = final_df['c_industries'].astype(str)
  final_df['top_c_industries'] = final_df['top_c_industries'].astype(str)
  return final_df

In [ ]:
# a function to convert the known country names to the continents they belong
# should first install pycountry and pycountry_convert
def country_to_continent(country_name):
  import pycountry_convert as pc
  if country_name == 'unknown':
    return 'unknown'
  else:
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name


### prepare dataset main function: clean_join(p_df, ob_df, s_df, oo_df)

In [ ]:
# clean each dataset and then join them
def clean_join(p_df, ob_df, s_df, oo_df):
  #global final_df, new_p_df, new_ob_df, new_s_df, new_oo_df
  # convert the data columns to strings
  p_df.loc[:, ['pointoftime', 'starttime']] = p_df.loc[:, ['pointoftime', 'starttime']].astype(str)
  ob_df.loc[:, ['pointoftime', 'starttime']] = ob_df.loc[:, ['pointoftime', 'starttime']].astype(str)
  s_df.loc[:, ['pointoftime', 'starttime']] = s_df.loc[:, ['pointoftime', 'starttime']].astype(str)
  oo_df.loc[:, ['pointoftime', 'starttime']] = oo_df.loc[:, ['pointoftime', 'starttime']].astype(str)
  ### remove the duplicates within p dataset - keep the last
  new_p_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'parent', \
                                    'pQID', 'parentLabel', 'parentcountryLabel', 'pindustries', 'proportion', \
                                    'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  new_p_df = pd.concat([new_p_df, clear_each_df(p_df)]).reset_index(drop=True)

  ### remove the duplicates within ob dataset - keep the last
  new_ob_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'ownedby', \
                                    'obQID', 'ownedbyLabel', 'ownedbycountryLabel', 'obindustries', 'proportion', \
                                    'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  new_ob_df = pd.concat([new_ob_df, clear_each_df(ob_df)]).reset_index(drop=True)

  ### remove the duplicates within s dataset - keep the last
  new_s_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'subsidiary', \
                                    'sQID', 'subsidiaryLabel', 'subsidiarycountryLabel', 'sindustries', 'proportion', \
                                    'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  new_s_df = pd.concat([new_s_df, clear_each_df(s_df)]).reset_index(drop=True)

  ### remove the duplicates within oo dataset - keep the last
  new_oo_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'ownerof', \
                                    'ooQID', 'ownerofLabel', 'ownerofcountryLabel', 'ooindustries', 'proportion', \
                                    'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  new_oo_df = pd.concat([new_oo_df, clear_each_df(oo_df)]).reset_index(drop=True)

  ### join together the dataframes into a new_df
  new_df = pd.DataFrame(columns=["pQID", "parent", "parent_country", "p_industries", "cQID", "child", \
                                "child_country", "c_industries", "p_proportion", "proportionofLabel", \
                                "pointoftime", "starttime", "endtime"])
  # Create an empty list to store the new rows
  new_p_rows = []
  # Loop over the new_p_df dataframe
  for i in range(len(new_p_df)):
      # Create a new row as a dictionary
      new_p_row = {'pQID': new_p_df['pQID'][i], 'parent': new_p_df['parentLabel'][i], 'parent_country': new_p_df['parentcountryLabel'][i], \
                'p_industries': new_p_df['pindustries'][i], 'cQID': new_p_df['QID'][i], \
                'child': new_p_df['itemLabel'][i], 'child_country': new_p_df['itemcountryLabel'][i], 'c_industries': new_p_df['industries'][i], \
                'p_proportion':new_p_df['proportion'][i], 'proportionofLabel':new_p_df['proportionofLabel'][i], \
                'pointoftime':new_p_df['pointoftime'][i], 'starttime':new_p_df['starttime'][i], 'endtime': new_p_df['endtime'][i]}
      # Convert the dictionary to a dataframe and append it to the list
      new_p_rows.append(pd.DataFrame(new_p_row, index=[0]))
  # Concatenate the list of dataframes with the original dataframe
  new_df = pd.concat([new_df] + new_p_rows, ignore_index=True)
  new_df['p_proportion'] = new_df['p_proportion'].fillna(2)
  # Create an empty list to store the new rows
  new_ob_rows = []
  for i in range(len(new_ob_df)):
      new_ob_row = {'pQID': new_ob_df['obQID'][i], 'parent': new_ob_df['ownedbyLabel'][i], 'parent_country': new_ob_df['ownedbycountryLabel'][i], \
                'p_industries': new_ob_df['obindustries'][i], 'cQID': new_ob_df['QID'][i], \
                'child': new_ob_df['itemLabel'][i], 'child_country': new_ob_df['itemcountryLabel'][i], 'c_industries': new_ob_df['industries'][i], \
                'p_proportion':new_ob_df['proportion'][i], 'proportionofLabel':new_ob_df['proportionofLabel'][i], \
                'pointoftime':new_ob_df['pointoftime'][i], 'starttime':new_ob_df['starttime'][i], 'endtime': new_ob_df['endtime'][i]}
      new_ob_rows.append(pd.DataFrame(new_ob_row, index=[0]))
  # Concatenate the list of dataframes with the original dataframe
  new_df = pd.concat([new_df] + new_ob_rows, ignore_index=True)
  # Create an empty list to store the new rows
  new_s_rows = []
  for i in range(len(new_s_df)):
      new_s_row = {'pQID': new_s_df['QID'][i], 'parent': new_s_df['itemLabel'][i], 'parent_country': new_s_df['itemcountryLabel'][i], \
                'p_industries': new_s_df['industries'][i], 'cQID': new_s_df['sQID'][i], \
                'child': new_s_df['subsidiaryLabel'][i], 'child_country': new_s_df['subsidiarycountryLabel'][i], 'c_industries': new_s_df['sindustries'][i], \
                'p_proportion':new_s_df['proportion'][i], 'proportionofLabel':new_s_df['proportionofLabel'][i], \
                'pointoftime':new_s_df['pointoftime'][i], 'starttime':new_s_df['starttime'][i], 'endtime': new_s_df['endtime'][i]}
      new_s_rows.append(pd.DataFrame(new_s_row, index=[0]))
  # Concatenate the list of dataframes with the original dataframe
  new_df = pd.concat([new_df] + new_s_rows, ignore_index=True)
  # Create an empty list to store the new rows
  new_oo_rows = []
  for i in range(len(new_oo_df)):
      new_oo_row = {'pQID': new_oo_df['QID'][i], 'parent': new_oo_df['itemLabel'][i], 'parent_country': new_oo_df['itemcountryLabel'][i], \
                'p_industries': new_oo_df['industries'][i], 'cQID': new_oo_df['ooQID'][i], \
                'child': new_oo_df['ownerofLabel'][i], 'child_country': new_oo_df['ownerofcountryLabel'][i], 'c_industries': new_oo_df['ooindustries'][i], \
                'p_proportion':new_oo_df['proportion'][i], 'proportionofLabel':new_oo_df['proportionofLabel'][i], \
                'pointoftime':new_oo_df['pointoftime'][i], 'starttime':new_oo_df['starttime'][i], 'endtime': new_oo_df['endtime'][i]}
      new_oo_rows.append(pd.DataFrame(new_oo_row, index=[0]))
  # Concatenate the list of dataframes with the original dataframe
  new_df = pd.concat([new_df] + new_oo_rows, ignore_index=True)
  new_df2 = new_df.drop_duplicates().reset_index(drop = True)

  # drop the duplicates from the joined dataframe with various rules defined by drop_dupl_joined() function
  final_df = drop_dupl_joined(new_df2)
  # clear up industries and find the top ones for classification. unknown ones: get description (also checks if its human)
  final_df = top_industries(final_df)

  # replace missing country values with "unknown"
  final_df["parent_country"] = final_df["parent_country"].replace("", np.nan)
  final_df["child_country"] = final_df["child_country"].replace("", np.nan)
  mask_p = final_df["parent_country"].isna()
  mask_c = final_df["child_country"].isna()
  final_df.loc[mask_p, "parent_country"] = "unknown"
  final_df.loc[mask_c, "child_country"] = "unknown"
  # apply the country_to_continent function to get the continents the companies are located
  final_df['parent_continent'] = final_df['parent_country'].apply(country_to_continent)
  final_df['child_continent'] = final_df['child_country'].apply(country_to_continent)
  return final_df


### user call clean_join(p_df, ob_df, s_df, oo_df)

In [ ]:
p_df = test['p_df'][-1]
ob_df = test['ob_df'][-1]
s_df = test['s_df'][-1]
oo_df = test['oo_df'][-1]

In [ ]:
f_df = clean_join(p_df, ob_df, s_df, oo_df)
f_df.head()

<ipython-input-37-c9084d13663d>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:, 'proportion'] = 4
<ipython-input-37-c9084d13663d>:46: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df2.loc[:, 'proportion'] = 4
<ipython-input-37-c9084d13663d>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,pQID,parent,parent_country,p_industries,cQID,child,child_country,c_industries,p_proportion,proportionofLabel,pointoftime,starttime,endtime,top_p_industries,top_c_industries,parent_continent,child_continent
0,Q478214,"Tesla, Inc.",United States of America,solar industry; automotive industry; battery i...,Q75327597,Gigafactory Berlin-Brandenburg,Germany,"Factory in Grünheide, Germany",3.0,NaN,nan,nan,nan,solar industry,other,North America,Europe
1,Q478214,"Tesla, Inc.",United States of America,solar industry; automotive industry; battery i...,Q28221306,Giga New York,United States of America,photovoltaic (PV) cell factory,3.0,NaN,nan,nan,nan,solar industry,other,North America,North America
2,Q478214,"Tesla, Inc.",United States of America,solar industry; automotive industry; battery i...,Q7705509,Tesla Fremont Factory,United States of America,automotive industry,3.0,NaN,nan,nan,nan,solar industry,automotive industry,North America,North America
3,Q478214,"Tesla, Inc.",United States of America,solar industry; automotive industry; battery i...,Q7555824,SolarCity,United States of America,solar industry,2.0,NaN,nan,2016-11-21T00:00:00Z,nan,solar industry,solar industry,North America,North America
4,Q478214,"Tesla, Inc.",United States of America,solar industry; automotive industry; battery i...,Q55642234,Giga Shanghai,People's Republic of China,electric vehicle factory,3.0,NaN,nan,nan,nan,solar industry,other,North America,Asia


In [ ]:
f_df.tail()

,pQID,parent,parent_country,p_industries,cQID,child,child_country,c_industries,p_proportion,proportionofLabel,pointoftime,starttime,endtime,top_p_industries,top_c_industries,parent_continent,child_continent
174,Q181114,Stellantis North America,United States of America,automotive industry,Q5114658,Chrysler Headquarters and Technology Center,United States of America,"building in Michigan, United States",3.0,NaN,nan,nan,nan,automotive industry,other,North America,North America
175,Q181114,Stellantis North America,United States of America,automotive industry,Q29610,Chrysler,United States of America,automotive industry,3.0,NaN,nan,nan,nan,automotive industry,automotive industry,North America,North America
176,Q833456,Rolls-Royce Power Systems,Germany,mechanical engineering,Q1880663,Woodward L'Orange GmbH,Germany,mechanical engineering,3.0,NaN,nan,nan,nan,mechanical engineering,mechanical engineering,Europe,Europe
177,Q833456,Rolls-Royce Power Systems,Germany,mechanical engineering,Q548006,MTU Friedrichshafen,Germany,automotive industry,3.0,NaN,nan,nan,nan,mechanical engineering,automotive industry,Europe,Europe
178,Q26966,Mercedes-AMG,Germany,automotive industry,Q552702,HWA AG,Germany,public company,3.0,NaN,nan,nan,nan,automotive industry,other,Europe,Europe


In [ ]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pQID               179 non-null    object 
 1   parent             179 non-null    object 
 2   parent_country     179 non-null    object 
 3   p_industries       179 non-null    object 
 4   cQID               179 non-null    object 
 5   child              179 non-null    object 
 6   child_country      179 non-null    object 
 7   c_industries       179 non-null    object 
 8   p_proportion       179 non-null    float64
 9   proportionofLabel  0 non-null      object 
 10  pointoftime        179 non-null    object 
 11  starttime          179 non-null    object 
 12  endtime            179 non-null    object 
 13  top_p_industries   179 non-null    object 
 14  top_c_industries   179 non-null    object 
 15  parent_continent   179 non-null    object 
 16  child_continent    179 non

### function and call: visualise_b2b_network(final_df)

In [ ]:
!pip install pyvis --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.9 MB/s eta 0:00:00


In [ ]:
def visualise_b2b_network(final_df):
  # import packages
  import pandas as pd
  from pyvis.network import Network
  import seaborn as sns
  import colorsys
  from collections import Counter
  import json

  # create network object
  net = Network(
      notebook = True,
      directed = True,            # directed graph
      bgcolor = "snow",          # background color of graph
      font_color = "navy",        # use navy for node labels
      cdn_resources = 'in_line',  # make sure Jupyter notebook can display correctly
      height = "1000px",          # height of chart
      width = "100%",             # fill the entire width
      select_menu=True,           # user can choose from campany name list
      filter_menu=True,           # user can search for colors - indicators for industries
      neighborhood_highlight=True, # clicking on a node highlights its connections and grays out others
      )

  # create lists of nodes and edges from dataframe columns
  nodes = []
  edges = []
  for parent, child, p_industry, c_industry, top_p_industry, top_c_industry, parent_country, parent_continent, \
      child_country, child_continent, proportion, endtime in zip(final_df['parent'], \
                                                                 final_df['child'], \
                                                                 final_df['p_industries'], \
                                                                 final_df['c_industries'], \
                                                                 final_df['top_p_industries'], \
                                                                 final_df['top_c_industries'], \
                                                                 final_df['parent_country'], \
                                                                 final_df['parent_continent'], \
                                                                 final_df['child_country'], \
                                                                 final_df['child_continent'], \
                                                                 final_df['p_proportion'], \
                                                                 final_df['endtime']):
      nodes.append((parent, p_industry, top_p_industry, parent_country, parent_continent))
      nodes.append((child, c_industry, top_c_industry, child_country, child_continent))
      edges.append((parent, child, proportion, endtime))

  # remove duplicate nodes
  #nodes = list(set(nodes))

  # create a set of unique industries
  industries = set(final_df['top_p_industries']).union(set(final_df['top_c_industries']))

  # get the number of unique industries
  n_colors = len(industries)

  # create a dictionary that maps each unique industry to a pastel color
  pastel_colors = sns.color_palette('muted', n_colors)

  # define a function that converts an rgb tuple to an rgba string with a pastel factor
  def rgb_to_rgba(rgb, pastel_factor):
      # convert rgb values to hls values
      h, l, s = colorsys.rgb_to_hls(*rgb)
      # increase lightness by multiplying with pastel factor
      l *= pastel_factor
      # convert hls values back to rgb values
      r, g, b = colorsys.hls_to_rgb(h, l, s)
      # normalize rgb values to be between 0 and 255
      r = int(r * 255)
      g = int(g * 255)
      b = int(b * 255)
      # add alpha value of 0.7
      a = 0.7
      # format rgba values as a string
      rgba = f'rgba({r}, {g}, {b}, {a})'
      return rgba

  group_color_map = {}
  for i, top_industry in enumerate(industries):
      # use the function with pastel_colors[i] and a pastel factor of 1.2
      group_color_map[top_industry] = rgb_to_rgba(pastel_colors[i], 1.2)

  # create a dictionary that maps each node to its degree
  node_degree_map = Counter()
  for edge in edges:
      node_degree_map[edge[0]] += 1
      node_degree_map[edge[1]] += 1

  # define a function that takes a degree and a scaling factor and returns a size for the node
  def degree_to_size(degree, scaling_factor):
      # multiply degree by scaling factor to get base size
      base_size = degree * scaling_factor
      # add minimum size to base size to ensure no node is too small
      min_size = 3
      final_size = base_size + min_size
      # return final size as an integer
      return int(final_size)

  # create a dictionary that maps each node to its size
  node_size_map = {}
  for node in node_degree_map:
      # use the function with node_degree_map[node] and a scaling factor of 5
      node_size_map[node] = degree_to_size(node_degree_map[node], 5)

  # define a different color for the highlighted nodes
  highlight_color = 'green'
  #node1 = root_companies[0]
  #node2 = root_companies[1]

  # create a set of unique countries and continents
  countries = set(final_df['parent_country']).union(set(final_df['child_country']))
  continents = set(final_df['parent_continent']).union(set(final_df['child_continent']))

  # add nodes and edges to network object
  for node, industry, top_industry, countries, continents in nodes:
      color = group_color_map[top_industry]
      # check if the node name is equal to node1 or node2 and use highlight_color if so
      #if node == root_companies:
      #    color = highlight_color
      #else:
      #    color = group_color_map[top_industry]
      # assign shapes for teh continents
      if continents == 'North America':
        node_shape = 'triangle'
      elif continents == 'South America':
        node_shape = 'triangleDown'
      elif continents == 'Europe':
        node_shape = 'star'
      elif continents == 'Africa':
        node_shape = 'diamond'
      elif continents == 'Asia':
        node_shape = 'square'
      elif continents == 'Australia':
        node_shape = 'ellipse'
      else:
        node_shape = 'dot'
      size = node_size_map[node]
      net.add_node(node, label=node, group=top_industry, color=color, shape=node_shape, title=node + ', '+ countries + ' (' + industry + ')', value=size)

  for e in edges:
      if e[2] > 0 and e[2] < 0.5:
          net.add_edge(e[0], e[1], title=str(e[2]), arrows={"to": True}, color='turquoise') #value=(e[2]/100), dashes=[6,10,1,10]
      elif e[2] >= 0.5 and e[2] <= 1:
          net.add_edge(e[0], e[1], title=str(e[2]), arrows={"to": True}, color='violet') #value=(e[2]/10),  dashes=False
      elif e[2] == 2:
          net.add_edge(e[0], e[1], title="parent", arrows={"to": True}, color='salmon') #dashes=[5,5]
      elif e[2] == 3:
          net.add_edge(e[0], e[1], title="unknown value", arrows={"to": True}, color='lime') # dashes=[1,10]
      elif e[2] == 4:
          net.add_edge(e[0], e[1], title="end date: "+str(e[3]), arrows={"to": True}, color='grey')


  net.repulsion(
      node_distance=100,
      central_gravity=0.2,
      spring_length=200,
      spring_strength=0.05,
      damping=0.09,
  )

  net.show_buttons(filter_='physics')
  #net.show_buttons(filter_=['nodes', 'edges', 'physics'])

  # show network graph
  return net.show('B2B_network_Wiki.html')

In [ ]:
visualise_b2b_network(f_df)

B2B_network_Wiki.html
